In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Constants
BASE_PATH = "Stanford3dDataset_v1.2_Aligned_Version"
PROCESSED_PATH = "Processed_Area"
GRID_SIZE = 0.2  # in meters
AREAS = [f"Area_{i}" for i in range(1, 7)]



In [8]:
# Load all obstacle point clouds in a room
def load_obstacles(annotation_dir):
    obstacle_pts = []
    for fname in os.listdir(annotation_dir):
        if fname.endswith(".txt") and not fname.startswith("floor") and not fname.startswith("ceiling"):
            try:
                pts = np.loadtxt(os.path.join(annotation_dir, fname))
                obstacle_pts.append(pts)
            except Exception as e:
                print(f"  - Skipped {fname}: {e}")
    return np.vstack(obstacle_pts) if obstacle_pts else np.empty((0, 6))

In [9]:
# Convert 3D points to 2D occupancy grid 
def create_occupancy_grid(points, grid_size):
    x, y = points[:, 0], points[:, 1]
    min_x, max_x = x.min(), x.max()
    min_y, max_y = y.min(), y.max()
    width = int(np.ceil((max_x - min_x) / grid_size))
    height = int(np.ceil((max_y - min_y) / grid_size))
    grid = np.zeros((height, width), dtype=np.uint8)
    for px, py in zip(x, y):
        gx = int((px - min_x) / grid_size)
        gy = int((py - min_y) / grid_size)
        if 0 <= gx < width and 0 <= gy < height:
            grid[gy, gx] = 1
    return grid


In [10]:

# Process all rooms
all_summaries = []

for area in AREAS:
    print(f"\n=== Processing {area} ===")
    area_path = os.path.join(BASE_PATH, area)
    grid_out_path = os.path.join(PROCESSED_PATH, f"{area}_grid")
    png_out_path = os.path.join(PROCESSED_PATH, f"{area}_png")

    os.makedirs(grid_out_path, exist_ok=True)
    os.makedirs(png_out_path, exist_ok=True)

    room_dirs = [d for d in os.listdir(area_path) if os.path.isdir(os.path.join(area_path, d))]

    for room in room_dirs:
        annotation_dir = os.path.join(area_path, room, "Annotations")
        if not os.path.exists(annotation_dir):
            continue

        print(f"Processing {room} in {area}")
        obstacles = load_obstacles(annotation_dir)
        if obstacles.shape[0] == 0:
            print(f" - No valid points in {room}")
            continue

        grid = create_occupancy_grid(obstacles, GRID_SIZE)

        # Save grid and PNG
        grid_path = os.path.join(grid_out_path, f"{room}_grid.npy")
        png_path = os.path.join(png_out_path, f"{room}_grid.png")
        np.save(grid_path, grid)

        plt.figure(figsize=(6, 6))
        plt.imshow(grid, cmap='gray_r')
        plt.title(f"{room} ({area})")
        plt.axis('off')
        plt.savefig(png_path, bbox_inches='tight', pad_inches=0)
        plt.close()

        all_summaries.append((area, room, grid.shape, grid_path, png_path))


=== Processing Area_1 ===
Processing conferenceRoom_2 in Area_1
Processing office_6 in Area_1
Processing office_1 in Area_1
Processing office_8 in Area_1
Processing office_30 in Area_1
Processing office_9 in Area_1
Processing office_7 in Area_1
Processing office_31 in Area_1
Processing office_25 in Area_1
Processing office_22 in Area_1
Processing office_14 in Area_1
Processing office_13 in Area_1
Processing hallway_7 in Area_1
Processing office_12 in Area_1
Processing office_15 in Area_1
Processing office_23 in Area_1
Processing office_24 in Area_1
Processing hallway_1 in Area_1
Processing pantry_1 in Area_1
Processing hallway_6 in Area_1
Processing hallway_8 in Area_1
Processing conferenceRoom_1 in Area_1
Processing copyRoom_1 in Area_1
Processing office_2 in Area_1
Processing office_5 in Area_1
Processing office_4 in Area_1
Processing office_3 in Area_1
Processing hallway_3 in Area_1
Processing hallway_4 in Area_1
Processing office_21 in Area_1
Processing office_19 in Area_1
Process

In [11]:
# Display summary
import pandas as pd
summary_df = pd.DataFrame(all_summaries, columns=["Area", "Room", "Grid Shape", "Grid Path", "PNG Path"])
display(summary_df)

,Area,Room,Grid Shape,Grid Path,PNG Path
0,Area_1,conferenceRoom_2,"(31, 30)",Processed_Area/Area_1_grid/conferenceRoom_2_gr...,Processed_Area/Area_1_png/conferenceRoom_2_gri...
1,Area_1,office_6,"(15, 22)",Processed_Area/Area_1_grid/office_6_grid.npy,Processed_Area/Area_1_png/office_6_grid.png
2,Area_1,office_1,"(15, 30)",Processed_Area/Area_1_grid/office_1_grid.npy,Processed_Area/Area_1_png/office_1_grid.png
3,Area_1,office_8,"(15, 30)",Processed_Area/Area_1_grid/office_8_grid.npy,Processed_Area/Area_1_png/office_8_grid.png
4,Area_1,office_30,"(31, 37)",Processed_Area/Area_1_grid/office_30_grid.npy,Processed_Area/Area_1_png/office_30_grid.png
...,...,...,...,...,...
266,Area_6,office_29,"(15, 30)",Processed_Area/Area_6_grid/office_29_grid.npy,Processed_Area/Area_6_png/office_29_grid.png
267,Area_6,office_16,"(15, 19)",Processed_Area/Area_6_grid/office_16_grid.npy,Processed_Area/Area_6_png/office_16_grid.png
268,Area_6,office_11,"(17, 19)",Processed_Area/Area_6_grid/office_11_grid.npy,Processed_Area/Area_6_png/office_11_grid.png
269,Area_6,office_27,"(15, 30)",Processed_Area/Area_6_grid/office_27_grid.npy,Processed_Area/Area_6_png/office_27_grid.png


In [12]:
# Save the summary table as a CSV file
csv_output_path = os.path.join(PROCESSED_PATH, "occupancy_grid_summary.csv")
summary_df.to_csv(csv_output_path, index=False)

print(f"Summary CSV saved to: {csv_output_path}")


Summary CSV saved to: Processed_Area/occupancy_grid_summary.csv
